In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
# import warnings
import warnings
# filter warnings
warnings.filterwarnings('ignore')

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers.convolutional import Conv2D, MaxPooling2D

# **Veriseti Yükleme**
*Bu bölümde verileri yükleyip görselleştiriyoruz*

In [ ]:
# Egitim verileri yüklenir 
train = pd.read_csv("../input/digitrecognizer/train.csv")
X_train, X_test, y_train, y_test = train_test_split(train.drop('label',axis = 1), train.label, 
                                                 test_size=10000, random_state=123, 
                                                 stratify=train.label)

X_train, X_test = X_train.to_numpy(), X_test.to_numpy()
print(train.shape)
train.head()

# Verilerin 28x28 Boyutunda Çizdirilmesi

In [ ]:
X_train = [np.reshape(x, (28,28)) for x in X_train]
X_test = [np.reshape(x, (28,28)) for x in X_test]
fig = plt.figure(figsize = (15,8))
for i, image in enumerate(X_train[:10]):
    ax = fig.add_subplot(2, 5, i+1)
    ax.set_xticks([]); ax.set_yticks([])
    ax.imshow(np.uint8(image))
plt.show()

In [ ]:
# Test verileri yüklenir
test= pd.read_csv("../input/digitrecognizer/test.csv")
print(test.shape)
test.head()

In [ ]:
# Egitimdeki labelları alıp Y_train elde edilir
Y_train = train["label"]
# Labelların cıkarılmış hali
X_train = train.drop(labels = ["label"],axis = 1) 

In [ ]:
# Veri dagılımları gözlemlenir(Mesela;1 sayısından kaç adet var, 2 sayısından kaç adet var)
plt.figure(figsize=(15,7))
g=sns.countplot(Y_train, palette="icefire")
plt.title("Rakamların dağılımları")
Y_train.value_counts()

# **Normalizasyon işlemleri**


In [ ]:
# Veriler normalize edilir(Gri skalada gösterilir)
X_train = X_train / 255.0
test = test / 255.0
print("x_train shape: ",X_train.shape)
print("test shape: ",test.shape)

In [ ]:
# 28x28x1 3D  4 boyutlu matris haline getirir(Reshape)
X_train = X_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)
print("x_train shape: ",X_train.shape)
print("test shape: ",test.shape)

In [ ]:
# Rakamları kod karşılıgına dönüstürür 
from keras.utils.np_utils import to_categorical 
Y_train = to_categorical(Y_train, num_classes = 10)

# **Verilerin %'lik oalarak kaçta kaçı test veya egitim verisi olduguna karar verilir:**
 test verisi %10
 
 egitim verisi %90


In [ ]:
# egitim ve dogrulama verileri fit edilir
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1, random_state=2)
print("x_train shape",X_train.shape)
print("x_test shape",X_val.shape)
print("y_train shape",Y_train.shape)
print("y_test shape",Y_val.shape)

In [ ]:
# Örnek gösterim
plt.imshow(X_train[2][:,:,0],cmap='gray')
plt.show()

# Model Oluşturma

In [ ]:
# Keras kütüphanesi metodları
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop,Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

model = Sequential()                        #Model oluşturma komutu
#
model.add(Conv2D(filters = 8, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))         #8 adet 5x5'lik filtre kullanılmıştır
model.add(MaxPool2D(pool_size=(2,2)))                                 # 2x2'lik maxpool kullanıldı
model.add(Dropout(0.25))
#
model.add(Conv2D(filters = 16, kernel_size = (3,3),padding = 'Same',  #16 adet 3x3'lük filtre kullanıldı
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))                  #2 basamak atlayarak pool'u gezdirdi
model.add(Dropout(0.25))
# fully connected
model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

In [ ]:
# Ögrenme oranı optimize edilir
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)

In [ ]:
# Modeli derleme
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [ ]:
epochs = 10 
batch_size = 250

# 

# Verileri Yeniden Üretme

Overfitting engelleniyor


In [ ]:
# veriyi yeniden üretme
datagen = ImageDataGenerator(
        featurewise_center=False,  
        samplewise_center=False,  
        featurewise_std_normalization=False,  
        samplewise_std_normalization=False,  
        zca_whitening=False,  
        rotation_range=5,                       #resmin rastgele dönüş oranı 5 derece
        zoom_range = 0.1,                       #resme %1'lik zoom yapılıyor
        width_shift_range=0.1,                  #resmi rastgele yatayda %1'lik oranda hareket ettirir
        height_shift_range=0.1,                 #resmi rastgele dikeyde %1'lik oranda hareket ettirir
        horizontal_flip=False,                  #resmi rastgele yatayda çevirir
        vertical_flip=False)                    #resmi rastgele dikeyde çevirir

datagen.fit(X_train)

In [ ]:
# Fit Modeli fit etme
history = model.fit_generator(datagen.flow(X_train,Y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (X_val,Y_val), steps_per_epoch=X_train.shape[0] // batch_size)

In [ ]:
# Test Kayıplarının Görselleştirilmesi 
plt.plot(history.history['val_loss'], color='g', label="doğrulama kaybı")
plt.title("Test Kayiplari")
plt.xlabel("Epoch Sayisi")
plt.ylabel("Kayiplar")
plt.legend()
plt.show()

In [ ]:

import seaborn as sns
Y_pred = model.predict(X_val)
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
Y_true = np.argmax(Y_val,axis = 1) 
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
f,ax = plt.subplots(figsize=(8, 8))
sns.heatmap(confusion_mtx, annot=True, linewidths=0.01,cmap="Reds",linecolor="gray", fmt= '.1f',ax=ax)
plt.xlabel("Tahmin edilen sayı")
plt.ylabel("Gerçek değer")
plt.title("Tahmin-Gerçek Tablosu")
plt.show()